In [1]:
#task h3.1: Сделать свое решение для этого соревнования https://www.kaggle.com/competitions/digit-recognizer
# Результат: опубликовать решение в разделе. После отправить мне через сайт ку решение и какой у вас получился скор

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Загрузка данных
train = pd.read_csv('C:/Users/vtako/Desktop/Project/DL homework 3/train.csv')
test = pd.read_csv('C:/Users/vtako/Desktop/Project/DL homework 3/test.csv')

# Разделение данных на X и y
y_train = train["label"]
X_train = train.drop(labels = ["label"], axis=1)

# Нормализация данных
X_train = X_train / 255.0
test = test / 255.0

# Изменение формы данных для CNN
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Преобразование меток в one-hot encoding
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Аугментация данных
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)
datagen.fit(X_train)

# Построение улучшенной модели CNN
model = Sequential()

# Первый сверточный блок
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='Same', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Второй сверточный блок
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Третий сверточный блок
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Переход к полносвязным слоям
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Компиляция модели
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callback-и
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# Обучение модели
epochs = 50
batch_size = 128

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    epochs=epochs, validation_data=(X_val, y_val),
    steps_per_epoch=X_train.shape[0] // batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Оценка модели
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Предсказания для тестового набора
predictions = model.predict(test)
predicted_classes = np.argmax(predictions, axis=1)

# Подготовка файла для отправки
submission = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_classes) + 1),
    'Label': predicted_classes
})

# Сохранение файла
submission.to_csv('C:/Users/vtako/Desktop/Project/DL homework 3/sample_submission.csv', index=False)


Epoch 1/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 73s 232ms/step - accuracy: 0.6970 - loss: 1.1074 - val_accuracy: 0.3876 - val_loss: 3.1554 - learning_rate: 0.0010
Epoch 2/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9062 - loss: 0.3265 - val_accuracy: 0.3612 - val_loss: 3.3011 - learning_rate: 0.0010
Epoch 3/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 60s 201ms/step - accuracy: 0.9584 - loss: 0.1402 - val_accuracy: 0.9855 - val_loss: 0.0503 - learning_rate: 0.0010
Epoch 4/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9766 - loss: 0.0989 - val_accuracy: 0.9869 - val_loss: 0.0483 - learning_rate: 0.0010
Epoch 5/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 58s 195ms/step - accuracy: 0.9708 - loss: 0.0966 - val_accuracy: 0.9902 - val_loss: 0.0401 - learning_rate: 0.0010
Epoch 6/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9766 - loss: 0.0806 - val_accuracy: 0.9902 - val_loss: 0.0400 - learning_rate: 0.0010
Epoch 7/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 53s 177ms/step - accuracy: 0.9759 - lo